In [12]:
import os
import pickle
import pandas as pd
import numpy as np
import nd2

from tifffile import imsave
from skimage import transform
import napari

import ipywidgets as widgets
from IPython.display import display

In [2]:
dir_path = r'Z:\Wayne\20210716_sen_sig_IF'
df_name = 'df_A3.pkl'
tmat_name = 'tmat_A3.pkl'

save_dir = r'D:\toDel\alig'

In [3]:
# read in the data
myData = pd.read_pickle(os.path.join(dir_path,df_name))

In [4]:
myData

,dir,sub_dir,file,channel_in_file,nameRound,well,signal,width,height,alignRound,width_min,height_min,alignIm
0,Z:\Wayne\20210716_sen_sig_IF,20210716_Round_1_DNA_bGal,"WellA3_ChannelHoechst,AF488_Seq1202.nd2",0.0,1.0,A3,DNA,16589.0,16590.0,0.0,16589.0,16590.0,1
1,Z:\Wayne\20210716_sen_sig_IF,20210716_Round_1_DNA_bGal,"WellA3_ChannelHoechst,AF488_Seq1202.nd2",1.0,1.0,A3,bGal,16589.0,16590.0,0.0,16589.0,16590.0,0
2,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",0.0,2.0,A3,DNA,16589.0,16590.0,1.0,16589.0,16590.0,1
3,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",1.0,2.0,A3,anti_1,16589.0,16590.0,1.0,16589.0,16590.0,0
4,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",2.0,2.0,A3,anti_2,16589.0,16590.0,1.0,16589.0,16590.0,0
5,Z:\Wayne\20210716_sen_sig_IF,20210725_Round_3_DNA_CDK4,"WellA3_ChannelHoechst,AF647_Seq0400.nd2",0.0,3.0,A3,DNA,16589.0,16590.0,2.0,16589.0,16590.0,1
6,Z:\Wayne\20210716_sen_sig_IF,20210725_Round_3_DNA_CDK4,"WellA3_ChannelHoechst,AF647_Seq0400.nd2",1.0,3.0,A3,CDK4,16589.0,16590.0,2.0,16589.0,16590.0,0


In [5]:
tmat_list = pickle.load(open(os.path.join(dir_path,tmat_name), "rb"))

In [6]:
len(tmat_list)

3

# Align and save images

In [10]:
myProgress = widgets.FloatProgress()
myProgress.min = 0
myProgress.max = len(myData)
myProgress.value = 0
display(myProgress)

test_im = []
for ind,my_signal in myData.iterrows():
    
    # set progress bar
    myProgress.value = ind
    
    # get a path to the image file
    file_path = os.path.join(my_signal.dir,my_signal.sub_dir,my_signal.file)

    # get a handle to the nd2 file
    myIm = nd2.ND2File(file_path)

    # choose the right frame
    dask_im = myIm.to_dask()
    im = dask_im[my_signal.channel_in_file,:,:]
    
    # get the transform
    tf = tmat_list[int(my_signal.alignRound)]
    
    # apply transform
    im_alig = transform.warp(im,tf,output_shape=im.shape)
    im_alig[im_alig<0] = 0
    
    test_im.append(im_alig)
    
    # save image
    imsave(os.path.join(save_dir,f'Round_{str(int(my_signal.nameRound)).zfill(2)}_well{my_signal.well}_{my_signal.signal}.tif'),im_alig.astype('uint16'))
    
myProgress.value = len(myData)

FloatProgress(value=0.0, max=6.0)

In [ ]:
# test alignment

viewer = napari.Viewer()
viewer.add_image(np.array(test_im),blending = 'additive')